In [0]:
# PyPi Tensorflow and utilities installation for Datalab
!pip install --upgrade pip
!pip uninstall -y tensorflow PyHamcrest google-api-core google-cloud-core grpcio google-cloud-monitoring datalab google-cloud-bigquery google-auth six requests protobuf
!pip install --upgrade PyHamcrest google-api-core google-cloud-core grpcio datalab google-cloud-bigquery google-cloud-monitoring==0.31.1 google-auth==1.9 setuptools==41.0.0 six==1.13.0 requests==2.21.0 protobuf==3.8.0 tensorflow==2.1.0
!pip install --upgrade matplotlib pandas h5py sklearn tqdm bleach pillow seaborn

In [0]:
#Import needed modules:
import numpy as np
import pandas as pd
import os
import shutil
from PIL import Image
from io import BytesIO

In [0]:
# import needed modules:

# general python utilities
import os
import platform
import random
import shutil
import sys
import time
import datetime
import functools
import itertools as it
import copy
import warnings
import pickle
warnings.simplefilter(action='ignore', category=FutureWarning)

# recommended Python3 version >= 3.5
print('Python version: {}'.format(platform.sys.version))

# data-science & processing tools
import numpy as np
import pandas as pd
import sklearn.metrics
import h5py

# progress bar
try:
    from tqdm.notebook import tqdm
except ImportError:
    from tqdm import tqdm_notebook as tqdm

# plotting utilities
import seaborn as sns
import matplotlib.pyplot as plt

# tensorflow
import tensorflow as tf
#import tensorflow.keras as K

# required TensorFlow version >= 2.0.0
tf_version = tf.__version__
print('TensorFlow version: {}'.format(tf_version))
assert int(tf_version[0]) >= 2, "Tensorflow version must be >= 2.0"

# seed random numbers for reproducibility
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

print('\nImports Complete.')

Python version: 3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 21:41:56) 
[GCC 7.3.0]
TensorFlow version: 2.1.0

Imports Complete.


In [0]:
#import datasets, which already have been normalized, and processed. These are the images to analyze.
#Each image is (224,224,3)
normal = pickle.load(open("Data/normal.pkl", "rb")) #Normal
covid = pickle.load(open("Data/covid.pkl", "rb")) #COVID
tb = pickle.load(open("Data/tb.pkl", "rb")) #TB
bp = pickle.load(open("Data/bpneumonia.pkl", "rb")) #Bacterial Pneumonia
vp = pickle.load(open("Data/vpneumonia.pkl", "rb")) #Bacterial Pneumonia
print(len(normal))
print(len(covid))
print(len(tb))
print(len(bp))
print(len(vp))
print(normal[0].shape)

485
212
394
400
400
(224, 224, 3)


In [0]:
#Create train, validation, and testing datasets using SKLearn built in.
from sklearn.model_selection import train_test_split
#Define Binary Data
X = np.concatenate([normal,covid], axis = 0)
Y = np.array([0]*len(normal) + [1]*len(covid)) #Let 0 be normal, COVID be 1.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)
print("Binary-class Data:")
print(X.shape)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

#Define Multi-class Data
X_m = np.concatenate([normal,covid,tb,vp,bp], axis = 0)
a = np.array([[1,0,0,0,0],]*len(normal))
b = np.array([[0,1,0,0,0],]*len(covid))
c = np.array([[0,0,1,0,0],]*len(tb))
d = np.array([[0,0,0,1,0],]*len(vp))
e = np.array([[0,0,0,0,1],]*len(bp))
Y_m = np.concatenate((a,b,c,d,e),axis=0)

X_mTrain, X_mTest, Y_mTrain, Y_mTest = train_test_split(X_m, Y_m, test_size=0.25, random_state=42)
X_mTrain, X_mVal, Y_mTrain, Y_mVal = train_test_split(X_mTrain, Y_mTrain, test_size=0.25, random_state=42)
print('Muli-class Data:')
print(X_m.shape)
print(X_mTrain.shape)
print(X_mVal.shape)
print(X_mTest.shape)
print(Y_mTest.shape)

Binary-class Data:
(697, 224, 224, 3)
(391, 224, 224, 3)
(131, 224, 224, 3)
(175, 224, 224, 3)
Muli-class Data:
(1891, 224, 224, 3)
(1063, 224, 224, 3)
(355, 224, 224, 3)
(473, 224, 224, 3)
(473, 5)


In [0]:
#import models
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications import vgg16, vgg19, resnet, resnet_v2, inception_v3, densenet
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, Dense, Dropout, Lambda

In [0]:
#VGG16
vgg16_base = vgg16.VGG16(weights = 'imagenet',
                    include_top = False,
                    input_shape = (224,224, 3))
#VGG19
vgg19_base = vgg19.VGG19(weights = 'imagenet',
                    include_top = False,
                    input_shape = (224,224, 3)) 
#ResNet 50 layers
resnet50_base = resnet.ResNet50(weights='imagenet',
                               include_top = False,
                               input_shape = (224,224,3))
#ResNet 101 layers
resnet101_base = resnet.ResNet101(weights='imagenet',
                               include_top = False,
                               input_shape = (224,224,3))
#ResNet 152 layers
resnet152_base = resnet.ResNet152(weights='imagenet',
                               include_top = False,
                               input_shape = (224,224,3))
##ResNetV2 50 layers
resnetV250_base = resnet_v2.ResNet50V2(weights='imagenet',
                               include_top = False,
                              input_shape = (224,224,3))
##ResNetV2 101 layers
resnetV2101_base = resnet_v2.ResNet101V2(weights='imagenet',
                               include_top = False,
                               input_shape = (224,224,3))
##ResNetV2 152 layers
resnetV2152_base = resnet_v2.ResNet152V2(weights='imagenet',
                               include_top = False,
                               input_shape = (224,224,3))

##DenseNets with 121, 169, 201 layers
densenet121 = tf.keras.applications.densenet.DenseNet121(weights = 'imagenet', include_top = False, input_shape=(224,224,3))
densenet169 = tf.keras.applications.densenet.DenseNet169(weights = 'imagenet', include_top = False, input_shape=(224,224,3))
densenet201 = tf.keras.applications.densenet.DenseNet201(weights = 'imagenet', include_top = False, input_shape=(224,224,3))

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint #to save best model.

Suggestions about how to integrate different convolutional models for transfer learning were based on work done by Adrian Xu: https://towardsdatascience.com/detecting-covid-19-induced-pneumonia-from-chest-x-rays-with-transfer-learning-an-implementation-311484e6afc1

#**Binary Classification Training and Testing**

In [0]:
#VGG16/19
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.0005

bases = [vgg19_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', #Use Binary CCE as loss.
                  optimizer=optimizers.Adam(lr=LEARNING_RATE), #use ADAM optimizer.
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_vgg16.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

In [0]:
#ResNet 50
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.005

bases = [resnet50_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(224, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_resnet50.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

Train on 391 samples, validate on 131 samples
Epoch 1/20
391/391 [==============================] - 68s 173ms/sample - loss: 7.4333 - acc: 0.6573 - val_loss: 0.6363 - val_acc: 0.7328
Epoch 2/20
391/391 [==============================] - 53s 134ms/sample - loss: 0.9786 - acc: 0.6624 - val_loss: 0.5892 - val_acc: 0.7328
Epoch 3/20
391/391 [==============================] - 53s 135ms/sample - loss: 0.7866 - acc: 0.6573 - val_loss: 0.6010 - val_acc: 0.7328
Epoch 4/20
391/391 [==============================] - 53s 134ms/sample - loss: 0.9645 - acc: 0.6471 - val_loss: 0.5999 - val_acc: 0.7328
Epoch 5/20
391/391 [==============================] - 52s 133ms/sample - loss: 0.7166 - acc: 0.6598 - val_loss: 0.5837 - val_acc: 0.7328
Epoch 6/20
391/391 [==============================] - 52s 133ms/sample - loss: 0.8963 - acc: 0.6419 - val_loss: 0.6019 - val_acc: 0.7328
Epoch 7/20
391/391 [==============================] - 53s 135ms/sample - loss: 2.2337 - acc: 0.6624 - val_loss: 0.8063 - val_acc: 0.

In [0]:
#ResNetV2 50
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.0005

bases = [resnetV250_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(224, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_resnetV250.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

Train on 391 samples, validate on 131 samples
Epoch 1/20
391/391 [==============================] - 59s 152ms/sample - loss: 0.6600 - acc: 0.8824 - val_loss: 32.9589 - val_acc: 0.7328
Epoch 2/20
391/391 [==============================] - 46s 116ms/sample - loss: 0.2730 - acc: 0.9182 - val_loss: 1728.7814 - val_acc: 0.7328
Epoch 3/20
391/391 [==============================] - 45s 116ms/sample - loss: 0.1600 - acc: 0.9642 - val_loss: 758.2625 - val_acc: 0.7328
Epoch 4/20
391/391 [==============================] - 45s 116ms/sample - loss: 0.1093 - acc: 0.9668 - val_loss: 365.7360 - val_acc: 0.7328
Epoch 5/20
391/391 [==============================] - 45s 115ms/sample - loss: 0.0983 - acc: 0.9821 - val_loss: 197.6758 - val_acc: 0.7328
Epoch 6/20
391/391 [==============================] - 50s 128ms/sample - loss: 0.0843 - acc: 0.9668 - val_loss: 93.5975 - val_acc: 0.7481
Epoch 7/20
391/391 [==============================] - 50s 128ms/sample - loss: 0.0170 - acc: 0.9923 - val_loss: 56.2088 -

In [0]:
#ResNetV2 101
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.0005

bases = [resnetV2101_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_resnetV2101.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

Train on 391 samples, validate on 131 samples
Epoch 1/20
391/391 [==============================] - 102s 261ms/sample - loss: 1.5299 - acc: 0.8363 - val_loss: 77.5671 - val_acc: 0.7328
Epoch 2/20
391/391 [==============================] - 82s 210ms/sample - loss: 0.3296 - acc: 0.8977 - val_loss: 313.7915 - val_acc: 0.7328
Epoch 3/20
391/391 [==============================] - 82s 209ms/sample - loss: 0.1110 - acc: 0.9668 - val_loss: 296.1700 - val_acc: 0.7328
Epoch 4/20
391/391 [==============================] - 82s 211ms/sample - loss: 0.1935 - acc: 0.9616 - val_loss: 376.8039 - val_acc: 0.7328
Epoch 5/20
391/391 [==============================] - 90s 231ms/sample - loss: 0.0379 - acc: 0.9923 - val_loss: 44.9194 - val_acc: 0.8244
Epoch 6/20
391/391 [==============================] - 91s 232ms/sample - loss: 0.0341 - acc: 0.9821 - val_loss: 26.6185 - val_acc: 0.8550
Epoch 7/20
391/391 [==============================] - 89s 229ms/sample - loss: 0.0618 - acc: 0.9821 - val_loss: 3.3867 - v

In [0]:
#ResNetV2 152
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.0005

bases = [resnetV2152_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(224, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_resnetV2152_extend.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

Train on 391 samples, validate on 131 samples
Epoch 1/20
391/391 [==============================] - 151s 387ms/sample - loss: 1.3997 - acc: 0.6496 - val_loss: 3996.2292 - val_acc: 0.7328
Epoch 2/20
391/391 [==============================] - 121s 310ms/sample - loss: 0.8400 - acc: 0.7059 - val_loss: 3.5440 - val_acc: 0.5115
Epoch 3/20
391/391 [==============================] - 133s 340ms/sample - loss: 0.4111 - acc: 0.8363 - val_loss: 313.1019 - val_acc: 0.7481
Epoch 4/20
391/391 [==============================] - 122s 311ms/sample - loss: 2.6930 - acc: 0.6394 - val_loss: 41719837.0687 - val_acc: 0.7328
Epoch 5/20
391/391 [==============================] - 121s 310ms/sample - loss: 3.4639 - acc: 0.6189 - val_loss: 1469.6208 - val_acc: 0.7328
Epoch 6/20
391/391 [==============================] - 122s 312ms/sample - loss: 1.0346 - acc: 0.6573 - val_loss: 63.1724 - val_acc: 0.7328
Epoch 7/20
391/391 [==============================] - 121s 310ms/sample - loss: 0.6860 - acc: 0.6496 - val_los

**Test DenseNet Versions**

In [0]:
#DenseNet
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.0005

bases = [densenet121]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_densenet121.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

Train on 391 samples, validate on 131 samples
Epoch 1/20
391/391 [==============================] - 95s 243ms/sample - loss: 2.4012 - acc: 0.8414 - val_loss: 52.5296 - val_acc: 0.7328
Epoch 2/20
391/391 [==============================] - 76s 195ms/sample - loss: 0.1907 - acc: 0.9412 - val_loss: 14.2003 - val_acc: 0.7328
Epoch 3/20
391/391 [==============================] - 80s 204ms/sample - loss: 0.0485 - acc: 0.9795 - val_loss: 1.1161 - val_acc: 0.7557
Epoch 4/20
391/391 [==============================] - 80s 205ms/sample - loss: 0.0155 - acc: 0.9923 - val_loss: 0.0557 - val_acc: 0.9771
Epoch 5/20
391/391 [==============================] - 80s 205ms/sample - loss: 0.0219 - acc: 0.9923 - val_loss: 0.0664 - val_acc: 0.9771
Epoch 6/20
391/391 [==============================] - 79s 201ms/sample - loss: 0.0098 - acc: 0.9949 - val_loss: 0.5559 - val_acc: 0.8015
Epoch 7/20
391/391 [==============================] - 79s 201ms/sample - loss: 0.0140 - acc: 0.9949 - val_loss: 0.2216 - val_acc: 

In [0]:
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.0005

bases = [densenet169]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_densenet169.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

Train on 391 samples, validate on 131 samples
Epoch 1/20
391/391 [==============================] - 112s 287ms/sample - loss: 3.9772 - acc: 0.7826 - val_loss: 22.2044 - val_acc: 0.7328
Epoch 2/20
391/391 [==============================] - 83s 214ms/sample - loss: 0.3039 - acc: 0.9028 - val_loss: 31.5372 - val_acc: 0.7328
Epoch 3/20
391/391 [==============================] - 88s 224ms/sample - loss: 0.1157 - acc: 0.9616 - val_loss: 9.2682 - val_acc: 0.8397
Epoch 4/20
391/391 [==============================] - 83s 213ms/sample - loss: 0.1310 - acc: 0.9540 - val_loss: 17.0698 - val_acc: 0.7328
Epoch 5/20
391/391 [==============================] - 82s 210ms/sample - loss: 0.0413 - acc: 0.9872 - val_loss: 6.2696 - val_acc: 0.7634
Epoch 6/20
391/391 [==============================] - 83s 211ms/sample - loss: 0.0645 - acc: 0.9795 - val_loss: 3.0692 - val_acc: 0.7786
Epoch 7/20
391/391 [==============================] - 87s 222ms/sample - loss: 0.0236 - acc: 0.9898 - val_loss: 0.0575 - val_acc

In [0]:
BATCH_SIZE = 20
NUM_EPOCHS = 20
LEARNING_RATE =0.0005

bases = [densenet201]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=7, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_densenet201.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_train,
                       y = Y_train,
                       validation_data = (X_val,Y_val),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                      callbacks=[earlyStopping, mcp_save])

In [0]:
#Load best models
saved_model152 = tf.keras.models.load_model('best_models/best_resnetV2152.hdf5')
saved_model101 = tf.keras.models.load_model("best_models/best_resnetV2101.hdf5")
saved_model50 = tf.keras.models.load_model("best_models/best_resnetV250.hdf5")
saved_model121=tf.keras.models.load_model('best_models/best_densenet121.hdf5')
saved_model169=tf.keras.models.load_model('best_models/best_densenet169.hdf5')
saved_model201=tf.keras.models.load_model('best_models/best_densenet201.hdf5')
saved_model16=tf.keras.models.load_model('best_models/best_vgg16.hdf5')
saved_model19=tf.keras.models.load_model('best_models/best_vgg19.hdf5')

In [0]:
#Get predictions on Test set.
y_pred_best50 = saved_model50.predict_classes(X_test)
y_pred_best101 = saved_model101.predict_classes(X_test)
y_pred_best152 = saved_model152.predict_classes(X_test)
y_pred_best121 = saved_model121.predict_classes(X_test)
y_pred_best169 = saved_model169.predict_classes(X_test)
y_pred_best201 = saved_model201.predict_classes(X_test)
y_pred_best16 = saved_model16.predict_classes(X_test)
y_pred_best19 = saved_model19.predict_classes(X_test)

In [0]:
#Get accuracy measurements using SKLearn.
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, y_pred_best50.reshape(-1)))
print(accuracy_score(Y_test, y_pred_best101.reshape(-1)))
print(accuracy_score(Y_test, y_pred_best152.reshape(-1)))
print(accuracy_score(Y_test, y_pred_best121.reshape(-1)))
print(accuracy_score(Y_test, y_pred_best169.reshape(-1)))
print(accuracy_score(Y_test, y_pred_best201.reshape(-1)))
print(accuracy_score(Y_test, y_pred_best16.reshape(-1)))
print(accuracy_score(Y_test, y_pred_best19.reshape(-1)))

0.9657142857142857
0.9942857142857143
0.96
0.9657142857142857
0.9657142857142857
0.8857142857142857
0.9485714285714286
0.9314285714285714


# **MULI-CLASS MODEL TRAINING**

**VGG Models**

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [vgg16_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_vgg16.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 320s 301ms/sample - loss: 1.3122 - acc: 0.4214 - val_loss: 1.0477 - val_acc: 0.5014
Epoch 2/30
1063/1063 [==============================] - 317s 299ms/sample - loss: 0.9481 - acc: 0.5663 - val_loss: 0.9583 - val_acc: 0.6310
Epoch 3/30
1063/1063 [==============================] - 332s 313ms/sample - loss: 0.8875 - acc: 0.6011 - val_loss: 0.7916 - val_acc: 0.6479
Epoch 4/30
1063/1063 [==============================] - 337s 317ms/sample - loss: 0.7026 - acc: 0.6914 - val_loss: 0.8646 - val_acc: 0.6056
Epoch 5/30
1063/1063 [==============================] - 343s 323ms/sample - loss: 0.6397 - acc: 0.7187 - val_loss: 0.7083 - val_acc: 0.6930
Epoch 6/30
1063/1063 [==============================] - 332s 313ms/sample - loss: 0.5648 - acc: 0.7563 - val_loss: 1.0546 - val_acc: 0.5803
Epoch 7/30
1063/1063 [==============================] - 323s 304ms/sample - loss: 0.5231 - acc: 0.7836 - val_loss

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [vgg19_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_vgg19.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 386s 363ms/sample - loss: 2.3200 - acc: 0.2295 - val_loss: 1.6493 - val_acc: 0.2169
Epoch 2/30
1063/1063 [==============================] - 387s 364ms/sample - loss: 1.6189 - acc: 0.2258 - val_loss: 1.5826 - val_acc: 0.2761
Epoch 3/30
1063/1063 [==============================] - 383s 360ms/sample - loss: 1.5937 - acc: 0.2399 - val_loss: 1.5815 - val_acc: 0.2085
Epoch 4/30
1063/1063 [==============================] - 398s 374ms/sample - loss: 1.5769 - acc: 0.2164 - val_loss: 1.5493 - val_acc: 0.2535
Epoch 5/30
1063/1063 [==============================] - 411s 387ms/sample - loss: 1.5012 - acc: 0.3302 - val_loss: 1.5907 - val_acc: 0.3014
Epoch 6/30
1063/1063 [==============================] - 414s 389ms/sample - loss: 1.6415 - acc: 0.2559 - val_loss: 1.5271 - val_acc: 0.2225
Epoch 7/30
1063/1063 [==============================] - 405s 381ms/sample - loss: 1.5453 - acc: 0.3029 - val_loss

**ResNet Models**  
Based on testing on binary models, we skipped testing ResNetV1, because it underperformed compared to V2.

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [resnetV250_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_resnetV250.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 123s 116ms/sample - loss: 3.7814 - acc: 0.4252 - val_loss: 13.1750 - val_acc: 0.2986
Epoch 2/30
1063/1063 [==============================] - 120s 113ms/sample - loss: 0.7732 - acc: 0.6482 - val_loss: 4.4707 - val_acc: 0.4254
Epoch 3/30
1063/1063 [==============================] - 123s 116ms/sample - loss: 0.5001 - acc: 0.7789 - val_loss: 3.9459 - val_acc: 0.4535
Epoch 4/30
1063/1063 [==============================] - 130s 122ms/sample - loss: 0.2662 - acc: 0.8956 - val_loss: 3.4135 - val_acc: 0.5099
Epoch 5/30
1063/1063 [==============================] - 122s 115ms/sample - loss: 0.1079 - acc: 0.9661 - val_loss: 4.8626 - val_acc: 0.4648
Epoch 6/30
1063/1063 [==============================] - 128s 121ms/sample - loss: 0.0890 - acc: 0.9680 - val_loss: 4.7075 - val_acc: 0.5831
Epoch 7/30
1063/1063 [==============================] - 125s 117ms/sample - loss: 0.1568 - acc: 0.9426 - val_los

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [resnetV2101_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_resnetV2101.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 234s 221ms/sample - loss: 4.1757 - acc: 0.4130 - val_loss: 120.5962 - val_acc: 0.1746
Epoch 2/30
1063/1063 [==============================] - 219s 206ms/sample - loss: 0.9237 - acc: 0.6077 - val_loss: 36.5577 - val_acc: 0.2592
Epoch 3/30
1063/1063 [==============================] - 210s 198ms/sample - loss: 0.6827 - acc: 0.6858 - val_loss: 37.5808 - val_acc: 0.2113
Epoch 4/30
1063/1063 [==============================] - 219s 206ms/sample - loss: 0.5249 - acc: 0.7742 - val_loss: 19.3112 - val_acc: 0.2873
Epoch 5/30
1063/1063 [==============================] - 221s 208ms/sample - loss: 0.3226 - acc: 0.8711 - val_loss: 10.1444 - val_acc: 0.3324
Epoch 6/30
1063/1063 [==============================] - 218s 206ms/sample - loss: 0.1881 - acc: 0.9323 - val_loss: 9.8727 - val_acc: 0.3437
Epoch 7/30
1063/1063 [==============================] - 218s 205ms/sample - loss: 0.1451 - acc: 0.9454 - va

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [resnetV2152_base]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_resnetV2152.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 338s 318ms/sample - loss: 3.2658 - acc: 0.4327 - val_loss: 7224.2168 - val_acc: 0.1859
Epoch 2/30
1063/1063 [==============================] - 304s 286ms/sample - loss: 0.8496 - acc: 0.6378 - val_loss: 1185.5992 - val_acc: 0.1859
Epoch 3/30
1063/1063 [==============================] - 316s 297ms/sample - loss: 0.5501 - acc: 0.7639 - val_loss: 48.8204 - val_acc: 0.2000
Epoch 4/30
1063/1063 [==============================] - 308s 290ms/sample - loss: 0.3956 - acc: 0.8410 - val_loss: 313.4380 - val_acc: 0.0901
Epoch 5/30
1063/1063 [==============================] - 322s 303ms/sample - loss: 0.1746 - acc: 0.9370 - val_loss: 7.3550 - val_acc: 0.3211
Epoch 6/30
1063/1063 [==============================] - 310s 292ms/sample - loss: 0.1614 - acc: 0.9492 - val_loss: 10.7557 - val_acc: 0.3014
Epoch 7/30
1063/1063 [==============================] - 341s 321ms/sample - loss: 0.1357 - acc: 0.9445 

**DenseNet Models**

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [densenet121]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_densenet121.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 200s 188ms/sample - loss: 3.5263 - acc: 0.6858 - val_loss: 6.6722 - val_acc: 0.6338
Epoch 2/30
1063/1063 [==============================] - 171s 160ms/sample - loss: 1.2030 - acc: 0.7940 - val_loss: 7.8190 - val_acc: 0.5070
Epoch 3/30
1063/1063 [==============================] - 171s 161ms/sample - loss: 1.3551 - acc: 0.8034 - val_loss: 5.7126 - val_acc: 0.5380
Epoch 4/30
1063/1063 [==============================] - 172s 162ms/sample - loss: 0.5834 - acc: 0.8768 - val_loss: 5.9962 - val_acc: 0.6169
Epoch 5/30
1063/1063 [==============================] - 171s 161ms/sample - loss: 0.2065 - acc: 0.9389 - val_loss: 10.0463 - val_acc: 0.5521
Epoch 6/30
1063/1063 [==============================] - 175s 165ms/sample - loss: 0.2071 - acc: 0.9577 - val_loss: 4.1603 - val_acc: 0.6507
Epoch 7/30
1063/1063 [==============================] - 172s 161ms/sample - loss: 0.1482 - acc: 0.9624 - val_los

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [densenet169]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_densenet169.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 252s 237ms/sample - loss: 8.0276 - acc: 0.3076 - val_loss: 2.6700 - val_acc: 0.3127
Epoch 2/30
1063/1063 [==============================] - 223s 210ms/sample - loss: 1.5155 - acc: 0.5550 - val_loss: 2.4263 - val_acc: 0.4000
Epoch 3/30
1063/1063 [==============================] - 213s 200ms/sample - loss: 0.9682 - acc: 0.6867 - val_loss: 4.3442 - val_acc: 0.2423
Epoch 4/30
1063/1063 [==============================] - 216s 203ms/sample - loss: 0.6502 - acc: 0.7667 - val_loss: 3.3862 - val_acc: 0.4592
Epoch 5/30
1063/1063 [==============================] - 207s 195ms/sample - loss: 0.5326 - acc: 0.7987 - val_loss: 3.6461 - val_acc: 0.3775
Epoch 6/30
1063/1063 [==============================] - 208s 196ms/sample - loss: 0.3595 - acc: 0.8664 - val_loss: 3.4419 - val_acc: 0.3606
Epoch 7/30
1063/1063 [==============================] - 208s 195ms/sample - loss: 0.2832 - acc: 0.9003 - val_loss

In [0]:
BATCH_SIZE = 80
NUM_EPOCHS = 30
LEARNING_RATE =0.0005

bases = [densenet201]
for base in bases:
    base.training = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation = "relu"))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=LEARNING_RATE),
                  metrics= ['acc'])
    earlyStopping = EarlyStopping(monitor="val_acc", patience=10, verbose=0, mode='max')
    mcp_save = ModelCheckpoint("best_models/best_multi_densenet201.hdf5", save_best_only=True, monitor='val_acc', mode = 'max')
    result = model.fit(x = X_mTrain,
                       y = Y_mTrain,
                       validation_data = (X_mVal,Y_mVal),
                       shuffle=True,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,callbacks=[earlyStopping, mcp_save])

Train on 1063 samples, validate on 355 samples
Epoch 1/30
1063/1063 [==============================] - 294s 277ms/sample - loss: 6.1193 - acc: 0.5146 - val_loss: 1247077.6197 - val_acc: 0.2986
Epoch 2/30
1063/1063 [==============================] - 257s 242ms/sample - loss: 1.8721 - acc: 0.6820 - val_loss: 30610.2322 - val_acc: 0.2225
Epoch 3/30
1063/1063 [==============================] - 269s 253ms/sample - loss: 0.8486 - acc: 0.7310 - val_loss: 70813.6311 - val_acc: 0.1972
Epoch 4/30
1063/1063 [==============================] - 267s 251ms/sample - loss: 0.5746 - acc: 0.7977 - val_loss: 19548.7437 - val_acc: 0.1972
Epoch 5/30
1063/1063 [==============================] - 283s 266ms/sample - loss: 0.4808 - acc: 0.7959 - val_loss: 3828.4940 - val_acc: 0.2789
Epoch 6/30
1063/1063 [==============================] - 296s 278ms/sample - loss: 0.3255 - acc: 0.8429 - val_loss: 1124.0873 - val_acc: 0.4423
Epoch 7/30
1063/1063 [==============================] - 295s 277ms/sample - loss: 0.2938 

In [0]:
#Load saved multi-class labels
saved_model152M = tf.keras.models.load_model('best_models/best_multi_resnetV2152.hdf5')
saved_model101M = tf.keras.models.load_model("best_models/best_multi_resnetV2101.hdf5")
saved_model50M = tf.keras.models.load_model("best_models/best_multi_resnetV250.hdf5")
saved_model121M=tf.keras.models.load_model('best_models/best_multi_densenet121.hdf5')
saved_model169M=tf.keras.models.load_model('best_models/best_multi_densenet169.hdf5')
saved_model201M=tf.keras.models.load_model('best_models/best_multi_densenet201.hdf5')
saved_model16M=tf.keras.models.load_model('best_models/best_multi_vgg16.hdf5')
saved_model19M=tf.keras.models.load_model('best_models/best_multi_vgg19.hdf5')

In [0]:
#Make test predictions
y_pred_best50M = saved_model50M.predict_classes(X_mTest)
y_pred_best101M = saved_model101M.predict_classes(X_mTest)
y_pred_best152M = saved_model152M.predict_classes(X_mTest)
y_pred_best121M = saved_model121M.predict_classes(X_mTest)
y_pred_best169M = saved_model169M.predict_classes(X_mTest)
y_pred_best201M = saved_model201M.predict_classes(X_mTest)
y_pred_best16M = saved_model16M.predict_classes(X_mTest)
y_pred_best19M = saved_model19M.predict_classes(X_mTest)

In [0]:
#Convert one-hot encoding to class-encoding.
Y_mTest=np.argmax(Y_mTest,1)

In [0]:
#Get Recall and Precision scores.
from sklearn.metrics import classification_report
print(classification_report(Y_mTest, y_pred_best121M.reshape(-1)))
print(classification_report(Y_mTest, y_pred_best169M.reshape(-1)))
print(classification_report(Y_mTest, y_pred_best201M.reshape(-1)))
print(classification_report(Y_mTest, y_pred_best50M.reshape(-1)))
print(classification_report(Y_mTest, y_pred_best101M.reshape(-1)))
print(classification_report(Y_mTest, y_pred_best152M.reshape(-1)))
print(classification_report(Y_mTest, y_pred_best16M.reshape(-1)))
print(classification_report(Y_mTest, y_pred_best19M.reshape(-1)))
#Get accuracy scores.
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_mTest, y_pred_best121M.reshape(-1)))
print(accuracy_score(Y_mTest, y_pred_best169M.reshape(-1)))
print(accuracy_score(Y_mTest, y_pred_best201M.reshape(-1)))
print(accuracy_score(Y_mTest, y_pred_best50M.reshape(-1)))
print(accuracy_score(Y_mTest, y_pred_best101M.reshape(-1)))
print(accuracy_score(Y_mTest, y_pred_best152M.reshape(-1)))
print(accuracy_score(Y_mTest, y_pred_best16M.reshape(-1)))
print(accuracy_score(Y_mTest, y_pred_best19M.reshape(-1)))

             precision    recall  f1-score   support

          0       0.78      0.56      0.65       124
          1       0.71      0.87      0.78        53
          2       0.66      0.70      0.68        98
          3       0.59      0.70      0.64       101
          4       0.64      0.62      0.63        97

avg / total       0.68      0.67      0.66       473

             precision    recall  f1-score   support

          0       0.71      0.81      0.75       124
          1       0.93      0.77      0.85        53
          2       0.73      0.74      0.74        98
          3       0.64      0.64      0.64       101
          4       0.66      0.59      0.62        97

avg / total       0.71      0.71      0.71       473

             precision    recall  f1-score   support

          0       0.72      0.77      0.74       124
          1       0.86      0.70      0.77        53
          2       0.69      0.71      0.70        98
          3       0.72      0.50      0